In [16]:
import sys
sys.path.insert(0, '../')
from utils import find_single_repetition, normalize_text, compute_string_similarity

In [7]:
import torch
import soundfile as sf
from IPython.display import display, Audio

In [18]:
from editor_app.database import SessionLocal
from editor_app.crud import get_project_by_title
from editor_app.stt import transcribe_utterance, stt_model
from tortoise.utils.wav2vec_alignment import Wav2VecAlignment

In [4]:
db = SessionLocal()

aligner = Wav2VecAlignment()

In [5]:
proj_title = 'timmermans_interview_demo'
user_id = 1
project = get_project_by_title(db, proj_title, user_id)

In [24]:
key_func = lambda x: x.date
for utter in project.utterances:
    stt_utterance = sorted(utter.utterance_stt, key=key_func)[-1]
    if stt_utterance.levenstein_similarity == 1.0:
        continue
    text_norm = normalize_text(utter.text)
    text_stt_norm = normalize_text(stt_utterance.text)
    res = find_single_repetition(text_stt_norm, text_norm)
    if not res:
        continue
    audio_path = utter.get_audio_path()
    wav, sample_rate = sf.read(audio_path, dtype='float32')
    display(Audio(data=wav, rate=sample_rate))
    wav = torch.from_numpy(wav).unsqueeze(0)
    res_wav = aligner.redact(wav, res)
    display(Audio(data=res_wav, rate=sample_rate))

    new_text_stt = stt_model.transcribe(res_wav.squeeze(0))['text']
    new_similarity_score = compute_string_similarity(utter.text, new_text_stt)
    print(stt_utterance.levenstein_similarity, new_similarity_score)


0.925 1.0


In [21]:
stt_model.transcribe(res_wav.squeeze(0))['text']

' This also shows a little bit of the desperation of the Russians. They are trying to blackmail us with that danger. That danger is there. That cannot be disregarded. It is very good that the United Nations, yet also courageous, is going there to see how things stand.'